In [1]:
from torch import optim
from lib.datasets import get_stock_price,sample_indices,train_test_split
from lib.aug import apply_augmentations,parse_augmentations
import torch
from torch import nn
import torch.nn.functional as F
from typing import List
from lib.utils import sample_indices


In [2]:
data_config = {
    "ticker" : "^GSPC",
    "interval" : "1d",
    "column" : 1,  
    "window_size" : 20,
    "dir" : "datasets",
    "subdir" : "stock"
}
sig_config = {
    "augmentations": [
        {"name": "AddTime"},
        {"name": "LeadLag"},
    ],
    "device" : "cuda:0",
    "depth" : 4,
}

In [3]:
tensor_data = get_stock_price(data_config)
x_real_train, x_real_test = train_test_split(tensor_data, train_test_ratio=0.8, device=sig_config["device"])
if sig_config["augmentations"] is not None:
    sig_config["augmentations"] = parse_augmentations(sig_config.get('augmentations'))
print("Before augmentation shape:",x_real_train.shape)
if sig_config["augmentations"] is not None:
    # Print the tensor shape after each augmentation
    x_aug_sig = apply_augmentations(x_real_train,sig_config["augmentations"])
    # Input dimension of encoder
    # We'll flat the tensor
    input_dim = x_aug_sig.shape[1]*x_aug_sig.shape[2]
print("After augmentation shape:",x_aug_sig.shape)
x_aug_sig = x_aug_sig.to(sig_config["device"])

Rolled data for training, shape torch.Size([1232, 20, 1])
Before augmentation shape: torch.Size([985, 20, 1])
torch.Size([985, 20, 2])
torch.Size([985, 39, 4])
After augmentation shape: torch.Size([985, 39, 4])


In [4]:
from models.betavae import BetaVAE

In [5]:
def train(model,optimizer):
    early_stop = 500
    cnt = 0
    min_loss = float('inf')
    for i in range(model.epoch):
        # Sample time indices of size equal to the batch size
        # From sefl.x_aug_sig
        time_indics = sample_indices(model.x_aug_sig.shape[0],model.batch_size,"cuda")
        sample_data = model.x_aug_sig[time_indics]
        # print("sample_data shape {}".format(sample_data.shape))
        # Encode 
        mean, log_var, z = model.encode(sample_data)
        # Decode
        reconstructed_data = model.decode(z)
        # print("reconstructed_data shape {},".format(reconstructed_data.shape))
        # Calculate loss
        loss = model.loss(mean,log_var,sample_data.view(sample_data.shape[0],-1),reconstructed_data)
        # Backpropogation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Print loss
        if i%500==0:
            print("Epoch {} loss {}".format(i,loss.item()))
        # Early stop
        if loss.item()<min_loss:
            min_loss = loss.item()
            cnt = 0
        else:
            cnt += 1
            if cnt>early_stop:
                break

In [6]:
lr = 1e-4
batch_size = 200
epoch = 20001
hidden_dims = [input_dim,12,3]
model = BetaVAE(x_aug_sig=x_aug_sig,epoch=epoch,batch_size=batch_size,hidden_dims=hidden_dims,device='cuda')
print(model)
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

Input tensor shape: torch.Size([985, 39, 4])
BetaVAE(
  (encoder_mu): Sequential(
    (0): Linear(in_features=156, out_features=12, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=12, out_features=3, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  )
  (encoder_sigma): Sequential(
    (0): Linear(in_features=156, out_features=12, bias=True)
    (1): Tanh()
    (2): Linear(in_features=12, out_features=3, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=12, out_features=156, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  )
)


In [7]:
train(model,optimizer=optimizer)

Epoch 0 loss 1388615168.0
Epoch 500 loss 620130048.0
Epoch 1000 loss 239009792.0
Epoch 1500 loss 117220048.0
Epoch 2000 loss 82478944.0
Epoch 2500 loss 65292888.0
Epoch 3000 loss 51641512.0
Epoch 3500 loss 31787168.0
Epoch 4000 loss 21586546.0
Epoch 4500 loss 16772200.0
Epoch 5000 loss 13256212.0
Epoch 5500 loss 11331558.0
Epoch 6000 loss 8656450.0
Epoch 6500 loss 7749748.0
Epoch 7000 loss 6225666.0
Epoch 7500 loss 5783278.5
Epoch 8000 loss 5003040.5
Epoch 8500 loss 4337531.5
Epoch 9000 loss 3917640.0
Epoch 9500 loss 3312057.25
Epoch 10000 loss 3073944.0
Epoch 10500 loss 2675198.75
Epoch 11000 loss 2348293.5
Epoch 11500 loss 2090581.75
Epoch 12000 loss 1896888.75
Epoch 12500 loss 1787290.5
Epoch 13000 loss 1591536.0
Epoch 13500 loss 1561827.5
Epoch 14000 loss 1546347.375
Epoch 14500 loss 1443865.0
Epoch 15000 loss 1275252.0
Epoch 15500 loss 1195807.125
Epoch 16000 loss 1124711.75
Epoch 16500 loss 1118846.125
